In [105]:
import black
import jupyter_black

jupyter_black.load(
    lab=False,
    line_length=79,
    target_version=black.TargetVersion.PY310,
)
from IPython.display import display, HTML, clear_output

display(HTML("<style>.container { width:100% !important; }</style>"))


from dataclasses import dataclass, field
import typing as t
import itertools as it
import collections as c
import json
from copy import deepcopy
import math as m
import time
import functools as ft
import numpy as np
import tqdm.notebook
import heapq

### Decimal to SNAFU

In [186]:
EXP = 5
NEG_MAP = {2: '=', 1: '-'}
SNAFU_MAP = {'=': -2, '-': -1, '0': 0, '1': 1, '2': 2}

In [198]:
def d2s(num, digits=None, exp=EXP, neg_map=NEG_MAP):
    # Terminal state
    end_st = {-2: '=', -1: '-', 0: 0, 1: 1, 2: 2}
    if num in end_st:
        digits[0] = end_st[num]
        return _dgt_lst2str(digits)
    neg_map = neg_map or NEG_MAP
    # Setup
    sign = -1 if num < 0 else 1
    absnum = abs(num)
    # Num digits
    k = m.log(absnum, exp)
    fk, ck = m.floor(k)+1, m.ceil(k)+1
    nd = ck if absnum > mx4digits(fk) else fk
    # equals for pows of EXP
    msd = 1 if fvpow(nd-1) + mx4digits(nd-1) >= absnum else 2
    digits = digits if digits else [0] * nd
    digits[nd-1] = msd if sign == 1 else neg_map[msd]
    new_num = num - sign * msd* fvpow(nd-1)
    # print(f"{num=}, {new_num=}, {fvpow(nd-1)=},{sign=}, {nd=}, {msd=}, {digits[nd-1]=}, {digits=}")
    d2s(new_num, digits=digits, exp=exp, neg_map=neg_map)
    return _dgt_lst2str(digits)

def _dgt_lst2str(lst):
    return ''.join(str(c) for c in reversed(lst))

@ft.cache
def mx4digits(n):
    return 2 *(EXP**n - 1) // (EXP-1)


@ft.cache
def fvpow(n):
    return 5**n

In [199]:
d2s(314159265)

'1121-1110-1=0'

### SNAFU to Decimal

In [189]:
def s2d(snafu, snafu_map=SNAFU_MAP):
    msd_pow = len(snafu) - 1
    dcml = 0
    for i, c in enumerate(snafu):
        dcml += snafu_map[c] * fvpow(msd_pow-i)
    return dcml

In [191]:
s2d("1121-1110-1=0")

314159265

In [192]:
def parse_fl(fl):
    return open(fl).read().splitlines()

In [201]:
print("Test:", d2s(sum(s2d(snfu) for snfu in parse_fl("test.txt"))))

Test: 2=-1=0


In [202]:
print("Input:", d2s(sum(s2d(snfu) for snfu in parse_fl("input.txt"))))

Input: 2-=2-0=-0-=0200=--21
